In [1]:
import time
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import pandas as pd


In [ ]:
options = webdriver.ChromeOptions()
# options.add_argument("--headless")  # retire o comentário para rodar sem abrir o navegador
driver = webdriver.Chrome(options=options)
wait = WebDriverWait(driver, 10)

In [ ]:
def extrair_info_artigo(link):
    driver.get(link)
    time.sleep(2)
    titulo = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'h1'))).text.strip()
    autores = []
    emails = []
    autores_links = driver.find_elements(By.CSS_SELECTOR, '[data-test="author-name"]')
    for autor in autores_links:
        nome = autor.text.strip()
        if nome:
            autores.append(nome)
            if 'icon-eds-i-mail-medium' in autor.get_attribute('innerHTML'):
                driver.execute_script("arguments[0].click();", autor)
                time.sleep(1)
                email_tag = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'a[href^="mailto:"]')))
                email = email_tag.get_attribute('href').replace('mailto:', '')
                if email and email not in emails:
                    emails.append(email)

    return [titulo, ', '.join(autores), ', '.join(emails)]

In [ ]:
def buscar_links_artigos(limit=5):
    driver.get("https://link.springer.com/search?query=&date=m24&journal=Automated+Software+Engineering")
    time.sleep(3)
    artigos = driver.find_elements(By.CSS_SELECTOR, 'a.app-card-open__link')
    print(f"🔍 {len(artigos)} artigos encontrados.")
    links = []
    for a in artigos:
        href = a.get_attribute('href')
        if href and '/article/' in href and href not in links:
            links.append(href)
        if len(links) >= limit:
            break

    return links

In [ ]:
dados = []
links = buscar_links_artigos(limit=5)

for i, link in enumerate(links):
    print(f"📄 ({i+1}/{len(links)}) Coletando dados do artigo: {link}")
    dados.append(extrair_info_artigo(link))
driver.quit()
if dados:
    with open('artigos_springer.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Título', 'Autores', 'Emails'])
        writer.writerows(dados)
    print("✅ Dados salvos em artigos_springer.csv")
else:
    print("⚠️ Nenhum dado foi extraído.")

In [22]:
df= pd.read_csv('dados_artigos_springer.csv')

In [23]:
df['Link'] = links

In [26]:
df.to_csv('dados_artigos_springer.csv')